In [ ]:
import random

In [ ]:
!pip install pymysql > /dev/null

from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving mysql.json to mysql.json


In [ ]:
import json
with open(filename) as fp:
    config_str = fp.read()
config = json.loads(config_str)

In [ ]:
import pymysql
conn = pymysql.connect(
    host = config['host'],
    user = config['user'],
    password = config['password'],
    database = config['database'],
    port = config['port']
)

# 테이블 생성

In [ ]:
# products 테이블 생성 sql
sql = """
    CREATE TABLE if NOT exists products(
        pid INT PRIMARY KEY AUTO_INCREMENT,
        pname VARCHAR(40) NOT NULL,
        pprice INT UNSIGNED NOT NULL,
        pcategory VARCHAR(20) NOT NULL,
        pcost INT UNSIGNED NOT NULL
    ) AUTO_INCREMENT=11;
"""

In [ ]:
cur = conn.cursor()
cur.execute(sql)

0

In [ ]:
camping_goods = ['텐트','타프','천막','침낭','매트','랜턴','취사용품',
                 '아이스박스','보조배터리','워터저그']
car_goods = ['블랙박스','내비게이션','하이패스','후방카메라','자동차TV',
             '카오디오','핸즈프리','헤드유닛','방진매트','윤활방청제']
health_goods = ['러닝머신','웨이트기구','복근운동기구','벨트','거꾸리',
                '훌라후프','트램펄린','스텝퍼','로잉머신','헬스사이클']
category_names = ['캠핑용품','자동차용품','헬스용품']

In [ ]:
for item in camping_goods:
    price = random.randint(100,999) * 100
    rate = random.randint(85, 98)
    cost = int(round(price * rate / 1000) * 10)
    print(item, price, '캠핑', cost)

텐트 32400 캠핑 28840
타프 63600 캠핑 55970
천막 49300 캠핑 46840
침낭 41200 캠핑 37490
매트 46000 캠핑 43700
랜턴 84300 캠핑 80080
취사용품 79000 캠핑 75050
아이스박스 77200 캠핑 67160
보조배터리 50300 캠핑 43760
워터저그 68600 캠핑 64480


In [ ]:
sql = "insert into products(pname,pprice,pcategory,pcost) values(%s,%s,%s,%s);"
for i, item_list in enumerate([camping_goods, car_goods, health_goods]):
    for item in item_list:
        price = random.randint(100,999) * 100
        rate = random.randint(85, 98)
        cost = int(round(price * rate / 1000) * 10)
        cur.execute(sql, (item,price,category_names[i],cost))
conn.commit()

## sales 테이블

In [ ]:
sql = """
    CREATE TABLE if NOT EXISTS sales(
        sid INT PRIMARY KEY AUTO_INCREMENT,
        sdate DATE NOT NULL,
        scompany VARCHAR(20) NOT NULL,
        spid INT NOT NULL,
        sunit INT NOT NULL,
        FOREIGN KEY(spid) REFERENCES products(pid)
    ) AUTO_INCREMENT=101;
"""

In [ ]:
date_list = []
calendar = {1:31,2:29,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
for k in range(1, 13):
    for i in range(random.randint(10, 13)):
        if k in (1, 3, 5, 6, 7, 10, 12):
            date = f'2020-{k}-{random.randint(1,31)}'
        elif k in (4, 6, 9, 11):
            date = f'2020-{k}-{random.randint(1,30)}'
        else:
            date = f'2020-{k}-{random.randint(1,29)}'
        date_list.append(date)


In [ ]:
cur = conn.cursor()
cur.execute(sql)

0

In [ ]:
sql = "insert into sales(sdate,scompany,spid,sunit) values(%s,%s,%s,%s);"
company_list = ['삼성', '엘지', '애플', '펜택', '구글', '소니']
for i in range(len(date_list)):
    sdate = date_list[i]
    scompany = random.choice(company_list)
    spid = random.randint(11, 40)
    sunit = random.randint(10, 100)
    
    cur.execute(sql, (sdate, scompany, spid, sunit))

conn.commit()


## 두 테이블을 Join 한 View 생성

In [ ]:
sql = """
    CREATE VIEW sales_book as
        SELECT sid, sdate, scompany, pid, pname, pcategory, pprice, sunit, pcost,
            pprice*sunit AS revenue, (pprice-pcost)*sunit AS profit
        FROM sales
        JOIN products
        ON sales.spid = products.pid
        ORDER BY sdate;
"""

In [ ]:
cur = conn.cursor()
cur.execute(sql)

0

- 데이터를 sales_book.csv로 저장

In [ ]:
sql = 'select * from sales_book'
cur.execute(sql)
results = cur.fetchall()

In [ ]:
import pandas as pd

df = pd.DataFrame(results)
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,21,2020-01-03,애플,40,헬스사이클,헬스용품,20600,72,18540,1483200,148320
1,23,2020-01-09,소니,40,헬스사이클,헬스용품,20600,90,18540,1854000,185400
2,16,2020-01-11,소니,38,스텝퍼,헬스용품,85300,46,83590,3923800,78660
3,24,2020-01-11,애플,23,하이패스,자동차용품,84200,47,77460,3957400,316780
4,18,2020-01-14,삼성,30,윤활방청제,자동차용품,59200,53,53870,3137600,282490


In [ ]:
del df[3]
del df[0]

In [ ]:
df.columns = ['판매일자','판매처','상품명','카테고리','단가','수량','원가','매출액','이익']
df.head()

,판매일자,판매처,상품명,카테고리,단가,수량,원가,매출액,이익
0,2020-01-03,애플,헬스사이클,헬스용품,20600,72,18540,1483200,148320
1,2020-01-09,소니,헬스사이클,헬스용품,20600,90,18540,1854000,185400
2,2020-01-11,소니,스텝퍼,헬스용품,85300,46,83590,3923800,78660
3,2020-01-11,애플,하이패스,자동차용품,84200,47,77460,3957400,316780
4,2020-01-14,삼성,윤활방청제,자동차용품,59200,53,53870,3137600,282490


In [ ]:
df.to_csv('sales_book.csv', index=False)

In [ ]:
cur.close()
conn.close()